In [ ]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter

import os
import sys

import pandas as pd
from PIL import Image

import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
!git --version

In [ ]:
!git config --global user.name "hajimins"
!git config --global user.email "minura.hajisoy@gmail.com"

In [ ]:
!git clone https://github.com/hajimins/YOLO-network-NN.git

In [ ]:
!mv /kaggle/working/your_file.ipynb YOLO-network-NN/

In [ ]:
!git add .